This notebook provides an implementation for the paper EFFICIENT AND SCALABLE GRAPH GENERATION THROUGH ITERATIVE LOCAL EXPANSION: [https://arxiv.org/pdf/2312.11529](https://arxiv.org/pdf/2312.11529)

In [189]:
!pip3 install -q torch_geometric
!pip3 install -q diffusers

# Import required librairies

In [190]:
import numpy as np
import math
from pprint import pprint

import torch
from torch.nn import Dropout, LayerNorm, Linear, Module, ModuleList
from torch.utils.data import IterableDataset
from torch.utils.data import DataLoader as torchDataLoader
import torch.nn.functional as F
from torch_geometric.utils import get_laplacian

from torch_geometric.nn import 
from torch_geometric.data import Data, Batch, DataLoader, InMemoryDataset, download_url 
from torch_geometric.utils import erdos_renyi_graph, to_dense_adj, dense_to_sparse, to_networkx, from_networkx
from torch_geometric.nn import GCNConv, GINConv

import networkx as nx
import matplotlib.pyplot as plt
import os

from diffusers import DDPMScheduler

SyntaxError: invalid syntax (2554226630.py, line 12)

In [ ]:
print("torch_geometric version:", torch_geometric.__version__)
print("torch version:", torch.__version__)

# Creating datasets for training

## Erdos renyi dataset

In [ ]:
def generate_erdos_renyi_graphData(num_nodes=5, p=0.6):
    edge_tensor = erdos_renyi_graph(num_nodes, p, directed=False)  # shape [2, num_edges]

    # Create graph nodes, values all 0
    nodes = torch.zeros((num_nodes, 1)) # shape [num_nodes, 1], any node feature is a single scalar 0.
    # each node in this tensor is assigned an id based on its index 0...num_nodes-1 
    # these ids are the ones used in edge_tensor, it is possible that there are no edges... 
    
    # Create Data object for graph
    graphData = Data(x=nodes, edge_index=edge_tensor)
    
    return graphData

graphDataset = generate_erdos_renyi_graphData()
# G = to_networkx(graph, to_undirected=True)
# plt.figure()
# nx.draw(G, with_labels=True, node_size=500)
# plt.show()

In [ ]:
# inspiration from https://pytorch-geometric.readthedocs.io/en/latest/tutorial/create_dataset.html#creating-in-memory-datasets

class ErdosRenyiDataset(InMemoryDataset):
    def __init__(self, root, num_graphs=100, transform=None, pre_transform=None, pre_filter=None):
        self.num_graphs = num_graphs

        super().__init__(root, transform, pre_transform, pre_filter)

        # Generate dataset if not done already, else load it
        processed_path = self.processed_paths[0]
        if not os.path.exists(processed_path):
            print("Creating dataset")
            self.process()
        else:
            print("Dataset already exists, loading it...")

        self.load(self.processed_paths[0]) # populates self.data
        print("Done.")

        """
        self.load will create objects:
        
        self.slices = {
          'x':          tensor([0, 2, 5, 6]),   # node feature offsets
          'edge_index': tensor([0, 1, 3, 4]),   # edge index offsets
        }

        self.data = {
          'x':          (total_num_nodes, num_node_features) # concatenation of all graphs nodes
          'edge_index': (2, total_num_edges) # concatenation of all graph edges
        }
        """

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list = []

        for i in range(self.num_graphs):
            data_list.append(generate_erdos_renyi_graphData())

        if self.pre_filter:
            data_list = [d for d in data_list if self.pre_filter(d)]
            
        if self.pre_transform:
            data_list = [self.pre_transform(d) for d in data_list]

        self.save(data_list, self.processed_paths[0]) # Runs: data, slices = cls.collate(data_list) and saves these objects

In [ ]:
!rm -rf /kaggle/working/data/ER/processed
erdos_renyi_dataset = ErdosRenyiDataset(root='data/ER', num_graphs=100)

In [ ]:
print(f"Number of graphs: {len(erdos_renyi_dataset)}")

for g in range(4):
    edge_index = erdos_renyi_dataset[g].edge_index

In [ ]:
edge_index[:, :1]

## Planar graph dataset (generated with plantri)

Command             | Triangulations Written | CPU Time (sec)
--------------------|------------------------|---------------
./plantri -c3 10    | 233                    | 0.00
./plantri -c3 11    | 1,249                  | 0.02
./plantri -c3 12    | 7,595                  | 0.04
./plantri -c3 13    | 49,566                 | 0.16
./plantri -c3 14    | 339,722                | 1.11
./plantri -c3 15    | 2,406,841              | 8.01
./plantri -c3 16    | 17,490,241             | 61.98
./plantri -c3 17    | 129,664,753            | 504.53

In [ ]:
import networkx as nx

def graph6_to_data(g6_path):
    print(f"Loading graphs from {g6_path}")
    
    with open(g6_path, "rb") as f:
        graph_nx = nx.read_graph6(f)

    print(f"Read {len(graphs_nx)} graphs from {g6_path}")
    return graphs_nx

In [ ]:
# inspiration from https://pytorch-geometric.readthedocs.io/en/latest/tutorial/create_dataset.html#creating-in-memory-datasets

class PlanarDataset(InMemoryDataset):
    def __init__(self, root, g6_dir=None, transform=None, pre_transform=None, pre_filter=None):

        g6_files = [os.path.join(g6_dir, f) for f in os.listdir(g6_dir) if f.endswith(".g6")]
        self.g6_files = g6_files
        
        super().__init__(root, transform, pre_transform, pre_filter)

        # Generate dataset if not done already, else load it
        processed_path = self.processed_paths[0]
        if not os.path.exists(processed_path):
            print("Creating dataset")
            self.process()
        else:
            print("Dataset already exists, loading it...")

        self.load(self.processed_paths[0]) # populates self.data
        print("Done.")

        """
        self.load will create objects:
        
        self.slices = {
          'x':          tensor([0, 2, 5, 6]),   # node feature offsets
          'edge_index': tensor([0, 1, 3, 4]),   # edge index offsets
        }

        self.data = {
          'x':          (total_num_nodes, num_node_features) # concatenation of all graphs nodes
          'edge_index': (2, total_num_edges) # concatenation of all graph edges
        }
        """

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list = []

        for file in self.g6_files:
            nx_graphs = graph6_to_data(file)
            data_list += [from_networkx(g) for g in nx_graphs]

        if self.pre_filter:
            data_list = [d for d in data_list if self.pre_filter(d)]
            
        if self.pre_transform:
            data_list = [self.pre_transform(d) for d in data_list]

        self.save(data_list, self.processed_paths[0]) # Runs: data, slices = cls.collate(data_list) and saves these objects

In [ ]:
#planar_dataset = PlanarDataset(root='data/PL', g6_dir="/kaggle/input/planar-graphs-g6-dataset")

# Coarsening sampling

The below function creates a dense adjency matrix given a graph by looking at all of its edges.

In [ ]:
def get_graph_adj(edge_index: torch.LongTensor, num_nodes: int):
    adj = to_dense_adj(edge_index, max_num_nodes=num_nodes)
    adj = adj[0]
    return adj

# test
test_graph_adj = get_graph_adj(test_graph.edge_index, test_graph.num_nodes)
#indices = (graph_adj == 1).nonzero(as_tuple=False)
print(test_graph_adj)

We need to sample coarsening sequences from $\Pi_F(G)$ that only use contraction sets in $F(G) = \mathcal{E}$, meaning each coarsening step only merges some edges of the graph. 

We sample in $\Pi_F(G)$ by sampling coarsening sequences from a "latent" distribution $q(\pi | G)$. To sample from $q(\pi | G)$, we associate a cost to each possible coarsening set at all coarsening steps, for now the uniform cost.

The below function extracts all possible coarsening sets at a given step (given a graph).

In [ ]:
def get_candidate_contraction_sets(G: Data):
    # make sure no self loop on nodes here!
    edge_index = G.edge_index # 2, num_edges (tensor)
    u, v       = edge_index[0], edge_index[1]

    # remove self edges
    nonself = (u != v)
    u_ns, v_ns = u[nonself], v[nonself]

    candidates = torch.stack([u_ns, v_ns], dim=1)
    return candidates

# test
test_graph = erdos_renyi_dataset[10]
nx.draw(to_networkx(test_graph), with_labels=True)

test_candidates = get_candidate_contraction_sets(test_graph)
print(test_candidates[:10])

test_current_num_nodes = test_graph.num_nodes
print("Total number of nodes:", test_current_num_nodes)

<span style="color:red">The uniform cost should be replaced by  local-variation cost of Loukas (2018) from the paper otherwise model won't train well!</span>

In [ ]:
def get_uniform_cost(candidate_contractions: torch.LongTensor, graphData: Data): # can also be based on G0, P1...P{l-1}
    # graphData not used, just used for compatibility
    num_candidates = candidate_contractions.shape[0]

    return torch.tensor([np.random.rand() for _ in range(num_candidates)], dtype=torch.float32)
      
# test
get_uniform_cost(test_candidates, test_graph)

Better cost function from the paper, written with chatgpt

In [ ]:
from torch.nn import Dropout, Linear, Module, ModuleList
from torch_geometric.nn import GINConv


class SignNet(Module):
    def __init__(
        self,
        num_eigenvectors: int,
        hidden_features: int,
        out_features: int,
        num_layers: int,
        dropout: float = 0.0,
    ) -> None:
        super().__init__()

        self.in_layer = Linear(2, hidden_features)
        self.conv_layers = ModuleList(
            [
                GINConv(
                    MLP(hidden_features, [hidden_features, hidden_features]),
                    train_eps=True,
                )
                for _ in range(num_layers)
            ]
        )
        self.skip_layer = Linear(hidden_features * (num_layers + 1), hidden_features)
        self.dropout = Dropout(dropout)
        # the following corresponds to the ρ function in the paper
        self.merge_layer = MLP(
            in_features=num_eigenvectors * hidden_features,
            hidden_features=[hidden_features, hidden_features],
            out_features=out_features,
        )

    def forward(self, spectral_features, edge_index):
        """Forward pass of the model.

        Args:
            spectral_features (Tensor): Eigenvalues (repeated) concatenated with eigenvectors. Shape: :math:`(V, num_eigenvectors * 2)`.
            edge_index (Adj): Adjacency matrix given as edge index or sparse tensor. Shape: :math:`(2, E)` or :math:`(V, V)`.

        Returns:
            Tensor: Node features. Shape: :math:`(V, out_features)`.
        """
        # Stack spectral features
        eigenvalues_repeated, eigenvectors = spectral_features.chunk(
            2, dim=-1
        )  # (V, k), (V, k)

        positive_spectral_features = torch.stack(
            [eigenvalues_repeated, eigenvectors], dim=-1
        )  # V, k, 2
        negative_spectral_features = torch.stack(
            [eigenvalues_repeated, -eigenvectors], dim=-1
        )  # V, k, 2
        combined_spectral_features = torch.stack(
            [positive_spectral_features, negative_spectral_features]
        ).transpose(
            1, 2
        )  # 2, k, V, 2

        # Apply layers
        x = self.in_layer(combined_spectral_features)  # 2, k, V, hidden_features
        xs = [x]
        for conv in self.conv_layers:
            # apply conv layer to each spectral feature independently
            x = conv(x=x, edge_index=edge_index)
            xs.append(x)

        # Skip connection
        x = torch.cat(xs, dim=-1)
        x = self.dropout(x)
        x = self.skip_layer(x)  # 2, k, V, hidden_features
        # Make sign invariant
        x = x.sum(dim=0)  # k, V, hidden_features

        # Merge features
        x = x.transpose(0, 1)  # V, k, hidden_features
        x = self.merge_layer(x.reshape(x.size(0), -1))  # V, out_features

        return x

In [ ]:
def get_spectral_features_torch(data, k):
    # Get Laplacian
    lap_edge_index, lap_edge_weight = get_laplacian(data.edge_index, normalization='sym')
    L_dense = 

    # Compute eigendecomposition
    eigvals, eigvecs = torch.linalg.eigh(L_dense)  # returns all eigenpairs

    # Take k smallest
    eigvals = eigvals[:k]                   # (k,)
    eigvecs = eigvecs[:, :k]                # (V, k)

    eigvals_repeated = eigvals.unsqueeze(0).repeat(data.num_nodes, 1)  # (V, k)
    spectral_features = torch.cat([eigvals_repeated, eigvecs], dim=-1)  # (V, 2k)

    return spectral_features
    

Below is the implementation of the rnd_greedy_min_cost_part algorithm 2 from the paper. Basically this algorithm runs at each step, and given number of nodes to remove, iteratively selectes the best contraction set (edges in our case) to use for coarsening, until the number of nodes to remove is obtained.

In [ ]:
def rnd_greedy_min_cost_part(
    candidate_contractions: torch.LongTensor, 
    cost_function, 
    current_graph, # added this parameter to compute cost, it contains current_graph on which coarsening is done
    num_nodes_to_remove: int, 
    current_num_nodes: int,
    rand_lambda: float = 1.0,
):
    """
    Algo 2
    candidate_contractions [num_candidates, 2] each candidate is a pair of nodes (edge)
    Greedily picks contraction candidates until at least num_nodes_to_remove are merged.
    """

    num_candidates = candidate_contractions.size(0)    
    alive = torch.ones(num_candidates, dtype=torch.bool) # check for already selected contractions, True means not selected
    marked  = torch.zeros(current_num_nodes, dtype=torch.bool) # mark nodes already contracted...
    
    costs  = cost_function(candidate_contractions, current_graph)
    
    # each cost is a rand float between 0 and 1    

    picked = [] # picked contractions idx
    removed = 0

    #print(f"inside rnd_greedy_min_cost_part: i need to remove {num_nodes_to_remove} nodes")
    while (removed < num_nodes_to_remove and alive.any()):
        masked_costs = costs.masked_fill(~alive, float('inf'))
        idx = torch.argmin(masked_costs).item()
        
        if torch.rand(1).item() > rand_lambda: # simulate Bernoulli, after selecting an edge to remove, sample a Bernoulli and while it is not 0 never select it again and continue process
            alive[idx] = False
            continue 

        selected_nodes = candidate_contractions[idx] # this contains the selected edge (pair of nodes) to remove
        
        if marked[selected_nodes].any(): # maybe one of the two selected nodes is already used in some other edge that needs to be removed, then never select edge again and continue process
            alive[idx] = False
            continue

        # otherwise mark it as selected set both of the nods to unalive and continue
        
        marked[selected_nodes] = True
        alive[idx] = False
        picked.append(idx)
        #print(f"We find a new edge to remove: {candidate_contractions[idx]}, removing it will remove: {selected_nodes.numel() - 1} nodes.")
        removed +=  selected_nodes.numel() - 1 # merging always keeps one node
        #print(f"inside rnd_greedy_min_cost_part: fo far i removed {removed} nodes.")

    #print(f"I might have stopped because alive.any(): {alive.any()}")
    if picked:
        picked_idx = torch.tensor(picked, dtype=torch.long)
        return candidate_contractions[picked_idx]
    else:
        print("rnd_greedy_min_cost_part Error")

# test
# for remove_n in range(1, current_num_nodes // 2): # max number of removable nodes
#     test_partitioning = rnd_greedy_min_cost_part(test_candidates, get_cost, remove_n, current_num_nodes)
#     print(f"Removed {remove_n} nodes with contractions: {test_partitioning}")
    
test_num_nodes_to_remove = 2
# uniform cost also possible with  get_uniform_cost instead of local_variation_costs
test_partitioning = rnd_greedy_min_cost_part(test_candidates, local_variation_costs, test_graph, test_num_nodes_to_remove, test_current_num_nodes)
print(test_partitioning)

Below is a visualization function that assigns a unique color to each cluster id (as long as there are less than 34 clusters in total), and then displays the graph by coloring clusters (all nodes in a given cluster will share that cluster color). 

In [ ]:
def visualize_clusters(G: Data, partitioning: torch.Tensor):
    # Convert to NetworkX
    if getattr(G, 'edge_attr', None) is not None:
        G_nx = to_networkx(G, to_undirected=True, edge_attrs=["edge_attr"])
        raw_attrs = {
            (u, v): int(d["edge_attr"])
            for u, v, d in G_nx.edges(data=True)
        }
    else:
        G_nx = to_networkx(G, to_undirected=True)
        raw_attrs = {}

    # Create a color map for each node
    color_map = ['lightgray'] * G.num_nodes  # default color

    # Assign a unique color for each cluster
    colors = [
        'red', 'green', 'blue', 'orange', 'purple', 'cyan', 'yellow', 'brown',
        'pink', 'olive', 'chocolate', 'lime', 'navy', 'teal', 'coral', 'gold',
        'magenta', 'orchid', 'salmon', 'darkgreen', 'deepskyblue', 'slateblue',
        'violet', 'maroon', 'turquoise', 'indigo', 'darkorange', 'crimson',
        'steelblue', 'darkviolet', 'khaki', 'cadetblue', 'hotpink', 'mediumseagreen'
    ]

    cluster_id_to_color = {}
    
    for cluster_id, cluster in enumerate(partitioning):
        color = colors[cluster_id % len(colors)]
        cluster_id_to_color[cluster_id] = color
        for node in cluster.tolist():
            color_map[node] = color
        print(f"Cluster {cluster_id} has color {colors[cluster_id % len(colors)]}")

    edge_colors = []
    for u, v in G_nx.edges():
        a = raw_attrs.get((u, v), None)
        edge_colors.append('red' if a == -1 else 'black')

    # Draw the graph
    pos = nx.spring_layout(G_nx, seed=42)
    nx.draw(G_nx, pos, node_color=color_map, with_labels=True, edge_color=edge_colors)
    plt.show()

    return cluster_id_to_color

In [ ]:
test_cluster_id_to_color = visualize_clusters(test_graph, test_partitioning)
print(f"Cluster 0 has color {test_cluster_id_to_color[0]}")

In [ ]:
def has_edge_undirected(edge_index, u, v):
    src, dst = edge_index
    return ((src == u) & (dst == v)).any() or ((src == v) & (dst == u)).any()

has_edge_undirected(test_graph.edge_index, 3, 5)

The below function completes a partitioning tensor containing all coarsening sets (pairs of nodes forming an edge that need to be merged) with the remaining sets that will not be merged for future code compatibility. The remaining sets not to be merged will all contain leftover nodes stacked with themselves, and concatenated at the end of the partitioning tensor. For instance one left over node i will be concatenated as [i, i].

In [ ]:
def augment_with_singletons(partitioning: torch.LongTensor, num_nodes: int, debug=False):
    """
    Given partitioning of shape [P,2]
    return a new LongTensor of shape [P+U, K'] where rows 0..P-1
    are your old clusters, and rows P..P+U-1 each contain exactly
    one leftover node as a single cluster.
    """
    # flatten and find which nodes were clustered
    clustered = partitioning.unique() # no dim specified -> unique of the flattened partitioning
    num_initial_clusters = partitioning.shape[0]
    
    all_nodes = torch.arange(num_nodes, device=partitioning.device)
    
    mask = torch.ones(num_nodes, dtype=torch.bool, device=partitioning.device)
    mask[clustered] = False
    
    leftover = all_nodes[mask]  # shape [U]

    # now make each leftover its own cluster of size 1, with itself, little trick
    singletons = torch.stack([leftover, leftover], dim=1) # [U, 2]
    
    if singletons.numel() > 0:
        partitioning = torch.cat([partitioning, singletons], dim=0)

        if debug:
            for i, node_id in enumerate(leftover.tolist()):
                cluster_id = num_initial_clusters + i
                print(f"Leftover node {node_id} assigned to singleton cluster {cluster_id}")

    return partitioning # ideally also explain what id each solo cluster got for coloring later!

# test 
print(f"Number of clusters before: {test_partitioning.shape[0]}")
test_augmented_partitioning = augment_with_singletons(test_partitioning, test_current_num_nodes, debug=True)
print(f"Number of clusters after: {test_augmented_partitioning.shape[0]}")

The below function builds a matrix P mapping all nodes to their clusters (left over nodes are considered as single node clusters) returns [num_clusters, N] for N nodes.

In [ ]:
def build_P(num_nodes: int,
            partitioning: torch.Tensor):
    """
    num_nodes: N
    partitioning: list of LongTensors, each contains original node‑indices cluster (max 2 nodes per cluster since we only consider edges).
    Returns P of shape [num_clusters, N], where P[c,i]=1 if i belongs to coarse‑node c.
    """
    num_clusters = len(partitioning)

    P = torch.zeros((num_clusters, num_nodes), dtype=torch.float32)
    for c, nodes in enumerate(partitioning):
        P[c, nodes] = 1.0
    return P

# test
test_P = build_P(test_current_num_nodes, test_augmented_partitioning)
print(test_P)

Each row corresponds to one cluster id, each column corresponds to one node. This can be reused to compute the expansion step GT as each cluster super node will need to be expanded back to however many nodes it contains.

The below function creates an adjency matrix for clusters. Two clusters which initially contained nodes connected to each other will end up connected. Self loops for one cluster are removed by filling the diagonal of this matrix with 0s.

In [ ]:
def get_cluster_adj(P: torch.FloatTensor,
                        A: torch.FloatTensor):
    # P: [C, N]; A: [N, N]
    M = P @ A @ P.t()       # [C, C], computes inter cluster links, if any two clusters initially had nodes connected they will be connected
    M.fill_diagonal_(0)
    return (M > 0).to(torch.float32)

# test
test_cluster_adj = get_cluster_adj(test_P, test_graph_adj)
print(test_cluster_adj)

The below function coarsens a graph given a partitioning containing coarsening sets (a set can be a cluster containing two nodes to merge, or a left over node copied twice that shouldn't be merged). It uses previous functions to returned the coarsened graph as well as the node-cluster mapping matrix that was used to build it.

In [ ]:
def coarsen_graph(G: Data, partitioning: torch.LongTensor):
    """
    Merge each row of partitioning into a single super‐node.
    Unclustered nodes are nodes j not appearing in any row of partitioning.
    Edges between any two super‐nodes exist iff any member nodes in both were previously connected in G.
    G{l}->G{l+1}
    """
    
    N = G.num_nodes

    augmented_partitioning = augment_with_singletons(partitioning, N)
    
    P = build_P(N, augmented_partitioning) # map nodes to clusters
    graph_adj = get_graph_adj(G.edge_index, N) # get initial graph adjency
    cluster_adj = get_cluster_adj(P, graph_adj) # get clusters adjency

    edge_index, edge_attr = dense_to_sparse(cluster_adj) # convert cluster adj to list of edges
    num_clusters = cluster_adj.size(0)
    
    x = torch.zeros((num_clusters, 1), dtype=torch.float32) # creates a list of cluster nodes
    return Data(x=x, edge_index=edge_index, num_nodes=num_clusters), P # returns a new graph using list of cluster nodes and how they should be connected + the node-cluster mapping matrix 

# test
test_coarsened_graph, test_P = coarsen_graph(test_graph, test_partitioning)

def get_coarse_node_colors(num_clusters: int, cluster_id_to_color: dict) -> list[str]:
    """
    Returns a list of colors for super-nodes in the coarsened graph.
    Each super-node corresponds to a cluster ID.
    """
    color_map = []
    for cluster_id in range(num_clusters):
        color = cluster_id_to_color.get(cluster_id, 'lightgray')
        color_map.append(color)
    return color_map

test_num_clusters = test_coarsened_graph.num_nodes
test_coarse_colors = get_coarse_node_colors(test_num_clusters, test_cluster_id_to_color)

# Step 3: Plot the coarsened graph
nx.draw(to_networkx(test_coarsened_graph), with_labels=True, node_color=test_coarse_colors)

A sequence of coarsening steps will always reach a single node graph eventually since the starting graph is fully connected and the only way to reach one node is to merge a graph containing two nodes (one edge).

In [ ]:
curr = 1
for i in range(10):
    print(curr)
    curr = curr-math.ceil(0.10*curr) # math.ceil(0.10*curr) represents the number of nodes to remove, it is computed in the algorithm that follows.

The below function implements rnd_red_seq (algorithm 1) from the paper. It takes as input the initial graph and returns the list of coarsened graphs at each step up to the final single node graph, as well as the contractions history (the node-cluster mappings for building each intermediate coarsened graph).

In [ ]:
def rnd_red_seq(G0: Data, min_red_frac=0.1, max_red_frac=0.3, debug=False): # it is not possible to remove more than current_num_nods//2 with edge contraction
    """
    Algo 1
    Returns random coarsening sequence π = (G0, . . . , GL) ∈ ΠF (G) 
    """

    current_graph = G0
    
    if debug: print(f"Initial graph has {G0.num_nodes} nodes.")
    coarsening_seq = [current_graph] # π, will end up containing (G0...GL)
    l = 0

    contractions_history = [] # will contain (P1...PL-1) where Pi is the partitioning matrix used to coarsen Gi into G{i+1}
    
    while current_graph.num_nodes != 1: # removing % of current_num_nods to current_num_nods will always be bigger than 0 and math.ceil 
        # will end up being one until current_num_nods becomes 1 -> last node will be removed 
        l += 1
        reduction_fraction = torch.tensor([1.0])
        reduction_fraction.uniform_(min_red_frac, max_red_frac)  # ρ

        # f already defined as get_cost, for now does not depend on G0/contractions_history
        num_nodes_to_remove = math.ceil(reduction_fraction * current_graph.num_nodes) # m

        candidates = get_candidate_contraction_sets(current_graph) # F(G{l−1}) fetch all possible contractions
        if candidates.numel() == 0:
            print("Error no more edges to contract in rnd_red_seq")
            break
        
        partitioning_l = rnd_greedy_min_cost_part(
            candidate_contractions=candidates,
            cost_function=get_cost,
            current_graph=current_graph,
            num_nodes_to_remove=num_nodes_to_remove,
            current_num_nodes=current_graph.num_nodes
        ) # tensor [C0, C1, …, Ck] # [num_clusters, 2]

        if partitioning_l.numel() == 0:
            print("No contractions picked")
            continue

        # coarsen graph
        if debug: print(f"Coarsening graph and trying to remove {num_nodes_to_remove} nodes.") # sometimes in some shapes such as S3 (star 4 nodes, one central) we cannot remove two nods
            # if we pick the center one...
        current_graph, P = coarsen_graph(current_graph, partitioning_l)
        if debug: print(f"Coarsened graph has {current_graph.num_nodes} nodes.")

        coarsening_seq.append(current_graph)
        contractions_history.append(P) 

    return coarsening_seq, contractions_history

# test
test_coarsening_seq, test_contractions_history = rnd_red_seq(test_graph, debug=True)
print(len(test_coarsening_seq), len(test_contractions_history))

print("\nCoarsening sequence:")
pprint(test_coarsening_seq)

print("\nContractions history:")
pprint(test_contractions_history)

# Retrieving data needed for expansion

In [ ]:
for g in erdos_renyi_dataset[0:5]:
    print(g)

The below function computes the cluster counts (v) given a node/cluster mapping matrix. Since we only use clusters of 1/2 nodes -> we reformulate the problem as classification between -1 if node shouldn't be expanded (cluster size 1) , 1 if node should be expanded (cluster size 2) 

In [ ]:
def compute_node_embeddings(P):
    cluster_counts = P.sum(dim=1).long() # v_l
    assign = P.argmax(dim=0)
    v = cluster_counts[assign]
    v = v-2
    return v

# test
print("P:")
print(test_P)

test_v = compute_node_embeddings(test_P)
print("Result:\n", test_v)

The below function expands a coarsened graph based on the node/cluster mapping matrix that was used to obtain that coarsened graph Gc, it returns the adjency matrix of the expanded graph.

In [ ]:
def expand_coarsened(Gc: Data, P):
    C, N = P.shape
    expanded_adj = np.zeros((N, N), dtype=int)
    
    coarsened_adj = get_graph_adj(Gc.edge_index, Gc.num_nodes) # [C, C]

    # for each node i find its cluster id
    clusters = P.argmax(dim=0)  # [N]  each in 0..C-1

    # # intercluster connections 
    # for i in range(N):
    #     for j in range(N):
    #         if clusters[i] != clusters[j]:
    #             ci, cj = clusters[i], clusters[j]
    #             if coarsened_adj[ci, cj] == 1:
    #                 expanded_adj[i, j] = 1
    #                 expanded_adj[j, i] = 1
    #         else:
    #             expanded_adj[i, j] = 1
    #             expanded_adj[j, i] = 1

    # np.fill_diagonal(expanded_adj, 0)
    # return expanded_adj
    
    # chatgpt optimized version using broadcasting
    idx_i = clusters.unsqueeze(1)  # [N,1]
    idx_j = clusters.unsqueeze(0)  # [1,N]

    # intercluster mask
    inter_mask = coarsened_adj[idx_i, idx_j].to(torch.int) # [N, N] 
    
    # intracluster mask
    intra_mask = idx_i == idx_j.to(torch.int) # [N, N]

    # combine and zero out diagonal no self loop
    expanded = (inter_mask | intra_mask)
    expanded.fill_diagonal_(False)

    return expanded
    
# test
test_expanded_graph_adj = expand_coarsened(test_coarsened_graph, test_P)
print(test_expanded_graph_adj)

<span style="color:red">Should we realy remove self loops here? expanded.fill_diagonal_(False) in the paper diagram for PPGN self loops are kept eventhough the model should learn to remove them of course...</span>

Below is the original test graph that we coarsen using the colored clusters.

In [ ]:
print("Original clustering")
test_cluster_id_to_color = visualize_clusters(test_graph, test_partitioning)

In [ ]:
test_cluster_id_to_color

Below is the coarsened graph obtained from the previous clusters + original graph.

In [ ]:
print("Original coarsened graph")
nx.draw(to_networkx(test_coarsened_graph), with_labels=True, node_color=test_coarse_colors)

Below is a function which given the adjency matrix of some expanded graph $\tilde{G}_l$  creates the refined graph Data object $G_l$.  $\tilde{G}_l$ was obtained after expanding some $\tilde{G}_{l+1}$ using $v_{l+1}$. The new graph should contain the same edges as the expanded graph, but with labels indicating whether we should keep (1) or remove/refine them (-1). All edges can be represented as $e_l$. Node embeddings will then contain for each node the future expansion counts required, all node embeddings can therefore be represented as $v_l$ later on.

In [ ]:
def refine_expanded_graph(expanded_adj: torch.Tensor, original_adj: torch.Tensor, v: torch.Tensor) -> Data:
    """
    expanded_adj: [N, N] dense adjacency (0/1)
    expanded_adj: [N, N] dense adjacency (0/1)
    
    returns: Data(x=[N,1], edge_index=[2, E])
    """
    edge_index, edge_attr = dense_to_sparse(expanded_adj)

    edge_labels = original_adj[edge_index[0],edge_index[1]].long() # [E]
    edge_labels = edge_labels * 2 - 1 # remap 0/1 to -1/1

    #x = torch.zeros((expanded_adj.size(0), 1), dtype=torch.float)
    x = torch.zeros((expanded_adj.size(0), 1), dtype=torch.float)
    y = v.float()

    return Data(x=x, edge_index=edge_index, edge_attr=edge_labels, y=y)

test_refinedGraphData = refine_expanded_graph(test_expanded_graph_adj, test_graph_adj, test_v)
print(test_refinedGraphData)
print(".x:", test_refinedGraphData.x)
print(".y:", test_refinedGraphData.y)
print("edge_attr:", test_refinedGraphData.edge_attr)

In [ ]:
print("Expanded and refined (red) graph coarsened graph")
test_cluster_id_to_color = visualize_clusters(test_refinedGraphData, test_partitioning)

We can extract from the refined graph the vector $e_l$ and for each edge print its -1/1 value.

In [ ]:
edge_index = test_refinedGraphData.edge_index   # shape [2, E]
edge_attr  = test_refinedGraphData.edge_attr    # shape [E] or [E,1]

for i in range(edge_index.size(1)):
    u = edge_index[0, i].item()
    v = edge_index[1, i].item()
    a = edge_attr[i].item() if edge_attr.ndim == 1 else edge_attr[i,0].item()
    print(f"Edge ({u:2d}, {v:2d})  → attr = {a}")

# Creating SignNet node embeddings from graph for conditional diffusion

This code is adapted from their implementation.

In [ ]:
class MLP(Module):
    """This block implements the multi-layer perceptron (MLP) module.

    Args:
        in_features (int): Number of features of the input.
        hidden_features (list[int]): List of the hidden features dimensions.
        out_features (int, optional): If not `None` a projection layer is added at the end of the MLP. Defaults to `None`.
        bias (bool, optional): Whether to use bias in the linear layers. Defaults to `True`.
        norm_layer (Module, optional): Normalization layer to use. Defaults to `norm_layer`.
    """

    def __init__(
        self,
        in_features: int,
        hidden_features: list[int],
        out_features: int | None = None,
        bias: bool = True,
        norm_layer=LayerNorm,
    ):
        super().__init__()
        lin_layers = []
        norm_layers = []
        hidden_in_features = in_features
        for hidden_dim in hidden_features:
            lin_layers.append(Linear(hidden_in_features, hidden_dim, bias=bias))
            norm_layers.append(norm_layer(hidden_dim))
            hidden_in_features = hidden_dim

        self.out_layer = (
            Linear(hidden_in_features, out_features, bias=bias)
            if out_features is not None
            else None
        )

        self.lin_layers = ModuleList(lin_layers)
        self.norm_layers = ModuleList(norm_layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        for lin, norm in zip(self.lin_layers, self.norm_layers):
            x = lin(x)
            x = norm(x)
            x = torch.relu(x)

        if self.out_layer is not None:
            x = self.out_layer(x)

        return x

In [ ]:
class SignNet(Module):
    def __init__(
        self,
        num_eigenvectors: int,
        hidden_features: int,
        out_features: int,
        num_layers: int,
        dropout: float = 0.0,
    ) -> None:
        super().__init__()

        self.in_layer = Linear(2, hidden_features)
        self.conv_layers = ModuleList(
            [
                GINConv(
                    MLP(hidden_features, [hidden_features, hidden_features]),
                    train_eps=True,
                )
                for _ in range(num_layers)
            ]
        )
        self.skip_layer = Linear(hidden_features * (num_layers + 1), hidden_features)
        self.dropout = Dropout(dropout)
        # the following corresponds to the ρ function in the paper
        self.merge_layer = MLP(
            in_features=num_eigenvectors * hidden_features,
            hidden_features=[hidden_features, hidden_features],
            out_features=out_features,
        )

    def forward(self, spectral_features, edge_index):
        """Forward pass of the model.

        Args:
            spectral_features (Tensor): Eigenvalues (repeated) concatenated with eigenvectors. Shape: :math:`(V, num_eigenvectors * 2)`.
            edge_index (Adj): Adjacency matrix given as edge index or sparse tensor. Shape: :math:`(2, E)` or :math:`(V, V)`.

        Returns:
            Tensor: Node features. Shape: :math:`(V, out_features)`.
        """
        # Stack spectral features
        eigenvalues_repeated, eigenvectors = spectral_features.chunk(
            2, dim=-1
        )  # (V, k), (V, k)

        positive_spectral_features = torch.stack(
            [eigenvalues_repeated, eigenvectors], dim=-1
        )  # V, k, 2
        negative_spectral_features = torch.stack(
            [eigenvalues_repeated, -eigenvectors], dim=-1
        )  # V, k, 2
        combined_spectral_features = torch.stack(
            [positive_spectral_features, negative_spectral_features]
        ).transpose(
            1, 2
        )  # 2, k, V, 2

        # Apply layers
        x = self.in_layer(combined_spectral_features)  # 2, k, V, hidden_features
        xs = [x]
        for conv in self.conv_layers:
            # apply conv layer to each spectral feature independently
            x = conv(x=x, edge_index=edge_index)
            xs.append(x)

        # Skip connection
        x = torch.cat(xs, dim=-1)
        x = self.dropout(x)
        x = self.skip_layer(x)  # 2, k, V, hidden_features
        # Make sign invariant
        x = x.sum(dim=0)  # k, V, hidden_features

        # Merge features
        x = x.transpose(0, 1)  # V, k, hidden_features
        x = self.merge_layer(x.reshape(x.size(0), -1))  # V, out_features

        return x

In [ ]:
def get_spectral_features_torch(data, k):
    # Get Laplacian
    lap_edge_index, lap_edge_weight = get_laplacian(data.edge_index, normalization='sym')
    L_dense = get_graph_adj(data.edge_index, data.num_nodes)

    # Compute eigendecomposition
    eigvals, eigvecs = torch.linalg.eigh(L_dense)  # returns all eigenpairs

    # Take k smallest
    eigvals = eigvals[:k]                   # (k,)
    eigvecs = eigvecs[:, :k]                # (V, k)

    eigvals_repeated = eigvals.unsqueeze(0).repeat(data.num_nodes, 1)  # (V, k)
    spectral_features = torch.cat([eigvals_repeated, eigvecs], dim=-1)  # (V, 2k)

    return spectral_features


# test
get_spectral_features_torch(test_graph, k=5)

In [ ]:
# inference

global_k = 32 # means the model takes as input 2x 32 = 64

V = test_graph.num_nodes
k_cur = min(global_k, V - 1)

embedding_model = SignNet(
    num_eigenvectors=global_k,
    hidden_features=64,
    out_features=128,
    num_layers=3,
    dropout=0.1
)

spectral_features = get_spectral_features_torch(test_graph, k=k_cur)
print(spectral_features.shape)

target_width = 2 * global_k
cur_width = spectral_features.shape[1]

if cur_width < target_width:
    pad_size = target_width - cur_width  # = 2*global_k − 2*k_cur
    padding = torch.zeros((V, pad_size))
    spectral_features = torch.cat([spectral_features, padding], dim=1)

print(spectral_features.shape)

embedding_model.eval()
with torch.no_grad():
    out = embedding_model(spectral_features, test_graph.edge_index)

print(out.shape)

In [ ]:
spectral_features.shape

In [ ]:
def fill_H_embeddings(G_coarsened: Data, G_refined: Data, P_l, embedding_model, v, k=0):
    num_nodes = G_coarsened.num_nodes
    if k==0:
        #H = torch.randn(num_nodes)
        x = torch.randn(2*k)
        G_refined.x = x
        return G_refined

    else:
        k_eig = min(k, num_nodes - 1)
        spectral_feats = get_spectral_features_torch(G_coarsened, k=k_eig)
    
        if k > k_eig:
            pad_cols = 2 * (k - k_eig)
            padding = torch.zeros((num_nodes, pad_cols))
            spectral_feats = torch.cat([spectral_feats, padding], dim=1)
    
        with torch.no_grad():
            H_coarsened = embedding_model(spectral_feats, G_coarsened.edge_index)

        print(H_coarsened.shape)

        # broadcast using P_l that was used to obtain G_coarsened from G
        cluster_id = P_l.argmax(dim=0)
        H_expanded = H_coarsened[cluster_id]

        G_refined.x = H_expanded
        
        return G_refined

# test
test_graph = fill_H_embeddings(test_coarsened_graph, test_graph, test_P, embedding_model, test_v, k=2)
print(test_graph.num_nodes)

# Creating the training dataset

The diffusion model will learn to denoise e_l (refine current graph), and v_l future expansions after refinement to get Gl-1, given the following parameters:

noised el and vl obtained from Gl˜ (noised T times using ddpm scheduler)
T
Gl˜ -> make diffusion conditional, that is the refined graph... Gl˜.x should contain the H embeddings replicated according to algo 5 in the paper, .y contains future expansion labels v -1/1, and edges attr contain -1/1
target size of G0?
something else?

It should output denoised el and vl (one step?)

In [ ]:
class CoarsenDataset(IterableDataset): # torch dataset
    def __init__(self, graphsDataset, min_red=0.1, max_red=0.3, start=0, end=None):
        super(CoarsenDataset).__init__()
        self.graphsDataset = graphsDataset
        self.start = start
        self.end = len(graphs_dataset) if end is None else end
        
    def __iter__(self):
        iter_start = self.start
        iter_end = self.end
        graphDataset = self.graphsDataset

        selected_graphs = graphDataset[iter_start: iter_end]
        
        for G0 in selected_graphs:
            coarsening_seq, contractions_history = rnd_red_seq(G0, self.min_red, self.max_red)
            # coarsening_seq = [G0, G1, ..., G_L]
            # contractions_history = [P1, P2, ..., P_{L}]  where P_l is used for G_{l-1}->G_l
            # P_l are the node-cluster mappings...

            num_initial_nodes = G0.num_nodes # target num nodes
            
            original_adj = get_graph_adj(G0.edge_index, G0.num_nodes)
            seq_len = len(coarsening_seq)
            print(seq_len)
            break
            
            for G_l, G_lp1, P_l in zip(coarsening_seq[:], coarsening_seq[1:], contractions_history):
                # to construct each sample we need:
                # from a sequence G{l-1} ---(Pl)---> Gl ----(Plp1)---> Glp1
                
                # first to expand Glp1 using Plp1 -> and obtain Gl˜ before refinement
                # compare Gl˜ and Gl to get refined graph and extract e_l -> add that to graph Gl˜ to obtain refined graph with proper edge attributes which is denoted as Gl
                # Use Pl to compute next node expansion for Gl (the expansion that will lead to G{l-1}˜)

                # for last node we know v=[2]
                # then given refined graphs + node embeddings/edgeattributes resp. G1,G2....,G{l-1} we can learn to create resp. G0, ..., Gl-2
                # we don't need to learn how to create Gl-1 since given any single node graph GL we just duplicate it into two nodes and that gives Gl-1...
                # therefore we can reconstruct any sequence backwards if model learns to reproduce the refinement+future expansion steps...

                coarsening_red_factor = G_lp1.num_nodes / G_l.num_nodes # reduction factor, proportion of nodes removed
                # during coarsening
                
                expanded_graph_adj = expand_coarsened(G_lp1, P_l) # Gl tilde, using actual cluster values
                v_l = compute_node_embeddings(P_l) # extract v_l counts of each cluster nodes
                # will be node embeddings

                refined_graph = refine_expanded_graph(expanded_graph_adj, original_adj, v_l) # data object, represents some G_l containing e_l to refine it... as well as future v_l labels in its .y
                ...
                
            
                yield 


def coarsen_collate(batch):


In [ ]:
import torch
from torch.utils.data import IterableDataset
from torch_geometric.data import Data, Batch

class CoarsenDataset(IterableDataset):
    """
    Yields training triples (graph, v_clean, e_clean, globals)
      graph.x       : [N, d_H+1]   –  concat(H , v_bin)
      graph.edge_attr: [E]         –  e_bin
      v_clean        : [N]         –  {-1,+1}
      e_clean        : [E]         –  {-1,+1}
      globals        : dict(ρ=float , target=int)
    """

    def __init__(
        self,
        graphs_dataset,                     # any PyG dataset
        signnet_model,                      # instantiated SignNet
        global_k      = 8,                  # eigenvectors for SignNet
        min_red       = 0.10,
        max_red       = 0.30,
        start         = 0,
        end           = None,
    ):
        super().__init__()
        self.graphs  = graphs_dataset
        self.model   = signnet_model
        self.k       = global_k
        self.min_red = min_red
        self.max_red = max_red
        self.start   = start
        self.end     = len(graphs_dataset) if end is None else end

    # ---------- helper -----------------------------------------------------
    def _broadcast_H(self, H_parent, P_l):          # P_l: [C,N]
        cluster_id = P_l.argmax(dim=0)              # [N]
        return H_parent[cluster_id]                 # [N, d_H]

    # ---------- iterator ---------------------------------------------------
    def __iter__(self):
        for G0 in self.graphs[self.start : self.end]:
            # 1. coarsen once per epoch
            seq, Ps = rnd_red_seq(G0, self.min_red, self.max_red)
            # seq = [G0,…, G_L]     Ps = [P1,…, P_L]

            target_size = G0.num_nodes
            for l in range(len(Ps)):                # iterate over levels
                G_l      = seq[l]                   # fine  graph at level l
                G_lp1    = seq[l+1]                 # coarse graph
                P_l      = Ps[l]                    # map nodes(G_l) -> nodes(G_{l+1})

                # 2. build expanded graph ẐG_l and labels
                expanded_adj = expand_coarsened(G_lp1, P_l)
                v_counts     = compute_node_embeddings(P_l)  # [N] (1/2)
                v_bin        = 2*v_counts - 3                # {-1,+1}

                e_bin_graph  = refine_expanded_graph(
                                    expanded_adj,
                                    get_graph_adj(G_l.edge_index, G_l.num_nodes),
                                    v_counts)
                e_bin = e_bin_graph.edge_attr                 # {-1,+1}

                # 3. SignNet embedding on G_{l+1}, then broadcast to ẐG_l
                H_parent = get_embeddings(G_lp1,
                                           self.model,
                                           v_counts,          # dummy arg
                                           k=self.k)          # [C, d_H]
                H_full   = self._broadcast_H(H_parent, P_l)   # [N, d_H]

                # 4. pack node features  [N, d_H+1]
                node_x = torch.cat([H_full, v_bin.unsqueeze(1).float()], dim=1)
                e_bin_graph.x         = node_x
                e_bin_graph.y_v_clean = v_bin          # supervision
                e_bin_graph.y_e_clean = e_bin

                # 5. globals
                rho = G_lp1.num_nodes / G_l.num_nodes  # reduction fraction
                globals = dict(rho=torch.tensor([rho], dtype=torch.float32),
                               target=torch.tensor([target_size], dtype=torch.long))

                yield e_bin_graph, globals


In [ ]:
def coarsen_collate(samples, scheduler, device="cpu"):
    """
    samples   : list[(Data, globals)]
    scheduler : your DDPM / VP-SDE scheduler with   .sample_timesteps(B)
                and   .add_noise(x, t)
    returns
        batch_graph         – PyG Batch with *noisy* node / edge features
        clean_v , clean_e   – tensors for the loss
        timesteps, globals  – tensors
    """
    graphs, globals = zip(*samples)
    batch    = Batch.from_data_list(graphs).to(device)

    # extract clean targets before adding noise
    clean_v = torch.cat([g.y_v_clean  for g in graphs]).to(device)  # [ΣN]
    clean_e = torch.cat([g.y_e_clean  for g in graphs]).to(device)  # [ΣE]

    # DDPM time-steps
    B        = len(graphs)
    t        = scheduler.sample_timesteps(B).to(device)             # [B]

    # node-wise noise
    node_feat = batch.x                                             # [ΣN,d+1]
    noisy_node_feat = scheduler.add_noise(node_feat, t.repeat_interleave(
                          batch.__slices__['x'][1:] - batch.__slices__['x'][:-1]))
    batch.x = noisy_node_feat

    # edge-wise noise
    edge_feat = batch.edge_attr.float().unsqueeze(1)                # [ΣE,1]
    noisy_edge_feat = scheduler.add_noise(edge_feat, t.repeat_interleave(
                          batch.__slices__['edge_index'][1:] - batch.__slices__['edge_index'][:-1]))
    batch.edge_attr = noisy_edge_feat.squeeze(1)

    # stack globals in the same order
    rhos    = torch.cat([g['rho']    for g in globals]).to(device)
    tgt_szs = torch.cat([g['target'] for g in globals]).to(device)

    return batch, clean_v, clean_e, t, rhos, tgt_szs


In [ ]:
dataset = CoarsenDataset(graphs_dataset=erdos_renyi_dataset,
                         signnet_model=embedding_model,
                         global_k=32)

loader  = torchDataLoader(dataset,
                          batch_size=32,
                          collate_fn=lambda b: coarsen_collate(b, scheduler),
                          num_workers=0)


In [ ]:
for ex in loader:
    print(ex)

In [ ]:
ds = CoarsenDataset(erdos_renyi_dataset, 0.1, 0.3, start=0, end=100)
loader = torchDataLoader(ds, batch_size=4, collate_fn=coarsen_collate)

for G_l, G_lp1, P_l in loader:
    print("G_l: ", G_l)
    print("G_lp1: ", G_lp1)
    print("P_l is of length: ", len(P_l))
    break

print("\nEach batch contains four graphs:")
for i in range(4):
    print(G_l.get_example(i))

# In a batch all nodes and edge_index are stacked but .batch keeps indices to filter for specific graphs

# Noising v_l and e_l for diffusion 

In [ ]:
scheduler = DDPMScheduler(num_train_timesteps=1000, beta_schedule="linear")
scheduler.set_timesteps(1000)

In [ ]:
v = torch.rand((4, 10))
t = torch.randint(0, 1000, (4,), dtype=torch.long)  # random timesteps per sample
noise = torch.randn_like(v)

In [ ]:
v_t = scheduler.add_noise(v, noise=noise, timesteps=t)
print(v_t)

In [ ]:
G_l.batch

In [ ]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class SimpleGCN(torch.nn.Module):
    def __init__(self, in_ch, hidden_ch, out_ch):
        super().__init__()
        self.conv1 = GCNConv(in_ch, hidden_ch)
        self.conv2 = GCNConv(hidden_ch, out_ch)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [ ]:
model = SimpleGCN(in_ch=1, hidden_ch=16, out_ch=8)
out = model(G_l)
G_l.x = out
graphs = G_l.to_data_list()

In [ ]:
graphs[0].x